In [1]:
import os, sys

In [2]:
'SUMO_HOME' in os.environ

True

In [3]:
tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
sys.path.append(tools)

In [4]:
sumoBinary = "C:/Program Files (x86)/DLR/Sumo/bin/sumo-gui"
sumoCmd = [sumoBinary, "-c", "C:/Users/sreeniva/Desktop/Reinforcement Learning/madrl_traffic_control/Sumo Stuff/hello.sumocfg"]

In [5]:
import traci

In [6]:
traci.start(sumoCmd)

(15, 'SUMO 0.30.0')

In [7]:
step=0
while step < 10:
   traci.simulationStep()
   step += 1

In [8]:
traci.trafficlights.getIDList()

['2']

In [1]:
import traci
import math

CAR_WIDTH = 5
MAX_HEIGHT = 200 / CAR_WIDTH
MAX_LENGTH = 200 / CAR_WIDTH
lane_ids = ["left-right-1_0", "left-right-2_0",
            "right-left-1_0", "right-left-2_0",
            "up-down-1_0", "up-down-2_0",
            "down-up-1_0", "down-up-2_0", ]


def initialize_matrix():
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    dtse_map = [[(vehicle_present, vehicle_vel, vehicle_id)
                 for x in range(MAX_LENGTH)] for y in range(MAX_HEIGHT)]
    return dtse_map


def get_lane_dimensions():
    # horizontal lane
    lane_width = traci.lane.getShape(lane_ids[0])[0][
        1] - traci.lane.getShape(lane_ids[3])[0][1]
    horizontal_x_min = 0.0
    horizontal_y_min = traci.lane.getShape(lane_ids[0])[0][1]
    horizontal_x_max = 200.0
    horizontal_y_max = horizontal_y_min + (2 * lane_width)

    # vertical lane
    vertical_x_min = traci.lane.getShape(lane_ids[4])[0][0]
    vertical_y_min = 0.0
    vertical_x_max = vertical_x_min + (2 * lane_width)
    vertical_y_max = 200.0

    return {'horizontal_x_min': horizontal_x_min / CAR_WIDTH,
            'horizontal_x_max': horizontal_x_max / CAR_WIDTH,
            'horizontal_y_min': horizontal_y_min / CAR_WIDTH,
            'horizontal_y_max': horizontal_y_max / CAR_WIDTH,
            'vertical_x_min': vertical_x_min / CAR_WIDTH,
            'vertical_x_max': vertical_x_max / CAR_WIDTH,
            'vertical_y_min': vertical_y_min / CAR_WIDTH,
            'vertical_y_max': vertical_y_max / CAR_WIDTH}


def print_matrix(matrix):
    output_str = ""
    for i in range(0, MAX_LENGTH):
        for j in range(0, MAX_HEIGHT):
            output_str += str(matrix[i][j]) + "\t"
        output_str += "\n"
    print output_str


def adjust_matrix_for_lanes(matrix):
    lane_dimensions = get_lane_dimensions()
    for x in range(0, MAX_LENGTH):
        for y in range(0, MAX_HEIGHT):
            if x > lane_dimensions.get('vertical_x_min') and x < lane_dimensions.get('vertical_x_max')\
                    and y > lane_dimensions.get('vertical_y_min') and y < lane_dimensions.get('vertical_y_max'):
                matrix[x][y][0] = 0
            if x > lane_dimensions.get('horizontal_x_min') and x < lane_dimensions.get('horizontal_x_max')\
                    and y > lane_dimensions.get('horizontal_y_min') and y < lane_dimensions.get('horizontal_y_max'):
                matrix[x][y][0] = 0

    return matrix


def get_left_right_dtse(x_min, x_max, y):
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    n_blocks = int(math.ceil(abs(x_max - x_min) / CAR_WIDTH))
    dtse_map = [[vehicle_present, vehicle_vel, vehicle_id] for x in range(n_blocks)]
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        (x_pos, y_pos) = traci.vehicle.getPosition(vehicle_id)
        vehicle_vel = traci.vehicle.getSpeed(vehicle_id)
        if x_pos > x_min and x_pos < x_max and y_pos == y:
            # make sure blocks are equally spaced starting from the junction
            block = (n_blocks -1) - int((x_max - x_pos) / CAR_WIDTH)
            # print x_pos, y_pos, block
            dtse_map[block] = [1, vehicle_vel, vehicle_id]
    return dtse_map

def get_right_left_dtse(x_min, x_max, y):
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    n_blocks = int(math.ceil(abs(x_max - x_min) / CAR_WIDTH))
    dtse_map = [(vehicle_present, vehicle_vel, vehicle_id) for x in range(n_blocks)]
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        (x_pos, y_pos) = traci.vehicle.getPosition(vehicle_id)
        vehicle_vel = traci.vehicle.getSpeed(vehicle_id)
        if x_pos > x_min and x_pos < x_max and y_pos == y:
            block = int((x_pos - x_min) / CAR_WIDTH)
            # print x_pos, y_pos, block
            dtse_map[block] = [1, vehicle_vel, vehicle_id]
    return dtse_map

def get_up_down_dtse(y_min, y_max, x):
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    n_blocks = int(math.ceil(abs(y_max - y_min) / CAR_WIDTH))
    dtse_map = [(vehicle_present, vehicle_vel, vehicle_id) for y in range(n_blocks)]
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        (x_pos, y_pos) = traci.vehicle.getPosition(vehicle_id)
        vehicle_vel = traci.vehicle.getSpeed(vehicle_id)
        if y_pos > y_min and y_pos < y_max and x_pos == x:
            # make sure blocks are equally spaced starting from the junction
            block = int((y_pos - y_min) / CAR_WIDTH)
            # print x_pos, y_pos, block
            dtse_map[block] = [1, vehicle_vel, vehicle_id]
    return dtse_map

def get_down_up_dtse(y_min, y_max, x):
    vehicle_vel = 0  # default
    vehicle_present = -1  # default
    vehicle_id = -999
    n_blocks = int(math.ceil(abs(y_max - y_min) / CAR_WIDTH))
    dtse_map = [(vehicle_present, vehicle_vel, vehicle_id) for y in range(n_blocks)]
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        (x_pos, y_pos) = traci.vehicle.getPosition(vehicle_id)
        vehicle_vel = traci.vehicle.getSpeed(vehicle_id)
        if y_pos > y_min and y_pos < y_max and x_pos == x:
            # make sure blocks are equally spaced starting from the junction
            block = int((y_max - y_pos) / CAR_WIDTH)
            # print x_pos, y_pos, block
            dtse_map[block] = [1, vehicle_vel, vehicle_id]
    return dtse_map

In [10]:
for lane in lane_ids:
    print lane, traci.lane.getShape(lane)

left-right-1_0 [(0.0, 48.35), (45.25, 48.35)]
left-right-2_0 [(54.75, 48.35), (100.0, 48.35)]
right-left-1_0 [(100.0, 51.65), (54.75, 51.65)]
right-left-2_0 [(45.25, 51.65), (0.0, 51.65)]
up-down-1_0 [(48.35, 100.0), (48.35, 54.75)]
up-down-2_0 [(48.35, 45.25), (48.35, 0.0)]
down-up-1_0 [(51.65, 0.0), (51.65, 45.25)]
down-up-2_0 [(51.65, 54.75), (51.65, 100.0)]


In [11]:
# left-right-1
[(x_min, y), (x_max, y1)] = traci.lane.getShape('left-right-1_0')
lr_1_dtse = get_left_right_dtse(x_min, x_max, y)

In [12]:
lr_1_dtse

[[-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [1, 4.0, 'type1.0'],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999],
 [-1, 0, -999]]

In [ ]:
x_min

In [ ]:
# right-left-1
[(x_max, y), (x_min, y1)] = traci.lane.getShape('right-left-1_0')
rl_1_dtse = get_left_right_dtse(x_min, x_max, y)
rl_1_dtse

In [ ]:
# left-right-2
[(x_min, y), (x_max, y1)] = traci.lane.getShape('left-right-2_0')
lr_2_dtse = get_left_right_dtse(x_min, x_max, y)

In [ ]:
lr_2_dtse

In [ ]:
traci.lane.getShape('up-down-1_0')

In [ ]:
[(x, y_max), (x1, y_min)] = traci.lane.getShape('up-down-1_0')
get_up_down_dtse(y_min, y_max, x)

In [ ]:
vehicle_ids = traci.vehicle.getIDList()

In [ ]:
# up-down-2
[(x, y_max), (x1, y_min)] = traci.lane.getShape('up-down-2_0')
ud_2_dtse = get_up_down_dtse(y_min, y_max, x)
ud_2_dtse

In [ ]:
traci.lane.getShape('down-up-1_0')

In [ ]:
# down-up-1
[(x, y_min), (x1, y_max)] = traci.lane.getShape('down-up-1_0')
du_1_dtse = get_down_up_dtse(y_min, y_max, x)
du_1_dtse

In [ ]:
from collections import defaultdict
vehicle_wait_times = defaultdict(lambda: defaultdict(lambda: 0.0))
min_speed = 0.1
# call this at every step
def get_avg_waiting_time():
    avg_wait_time = 0.0
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        if traci.vehicle.getSpeed(vehicle_id) < 0.1:
            vehicle_wait_times[vehicle_id]['t_wait'] += 1
        else:
            vehicle_wait_times[vehicle_id]['t_move'] += 1

        vehicle_wait_times[vehicle_id]['avg_wait_time'] =\
            vehicle_wait_times[vehicle_id]['t_wait'] / (vehicle_wait_times[vehicle_id]['t_wait'] + vehicle_wait_times[vehicle_id]['t_move'])

        # makes sure we don't include avg wait time from vehicles no longer on the network
        avg_wait_time +=  vehicle_wait_times[vehicle_id]['avg_wait_time']
    avg_wait_time = avg_wait_time / len(vehicle_ids)

    return avg_wait_time

In [ ]:
get_avg_waiting_time()

In [ ]:
vehicle_wait_times

In [ ]:
total_waiting_time = 0
total_moving_time = 0
def get_avg_waiting_time_v2():
    global total_waiting_time, total_moving_time
    vehicle_ids = traci.vehicle.getIDList()
    for vehicle_id in vehicle_ids:
        if traci.vehicle.getSpeed(vehicle_id) < 0.1:
            total_waiting_time += 1
        else:
            total_moving_time += 1
    
    avg_wait_time = 1.0 * total_waiting_time / (total_waiting_time + total_moving_time)
    return avg_wait_time

In [ ]:
get_avg_waiting_time_v2()

In [10]:
x = traci.trafficlights.getIDList()[0]

In [8]:
def run_sim(steps):
    step = 0
    while step < steps:
       traci.simulationStep()
       step += 1

In [13]:
traci.trafficlights.getPhase(x)
step = 0

In [14]:
prev_phase = 0
print prev_phase
counter = 0

while counter < 60:
    traci.simulationStep()
    if traci.trafficlights.getPhase(x) != prev_phase:
        print "Change! at step", step
        print "{} --> {}".format(prev_phase, traci.trafficlights.getPhase(x))
        prev_phase = traci.trafficlights.getPhase(x)
    step += 1
    counter += 1

0
Change! at step 0
0 --> 3
Change! at step 5
3 --> 4
Change! at step 9
4 --> 5
Change! at step 13
5 --> 6
Change! at step 44
6 --> 7
Change! at step 48
7 --> 8
Change! at step 52
8 --> 9


In [15]:
counter = 0
while counter < 60:
    traci.simulationStep()
    if traci.trafficlights.getPhase(x) != prev_phase:
        print "Change! at step", step
        print "{} --> {}".format(prev_phase, traci.trafficlights.getPhase(x))
        prev_phase = traci.trafficlights.getPhase(x)
    step += 1
    counter += 1

Change! at step 83
9 --> 10
Change! at step 87
10 --> 11
Change! at step 91
11 --> 0


In [ ]:
counter = 0
while counter < 60:
    traci.simulationStep()
    if traci.trafficlights.getPhase(x) != prev_phase:
        print "Change! at step", step
        print "{} --> {}".format(prev_phase, traci.trafficlights.getPhase(x))
        prev_phase = traci.trafficlights.getPhase(x)
    step += 1
    counter += 1

In [ ]:
traci.trafficlights.getRedYellowGreenState(x)

In [ ]:
traci.trafficlights.getPhase(x)

In [ ]:
traci.trafficlights.setPhase(x, 2)

In [9]:
run_sim(1000)

In [ ]:
traci.trafficlights.setRedYellowGreenState(x, 'GGGGrrrrrrrrrrrr')

In [ ]:
traci.trafficlights.getPhaseDuration(x)

In [ ]:
traci.trafficlights.setPhaseDuration(x, 1.0)

In [ ]:
traci.trafficlights.getRedYellowGreenState(x)

In [11]:
traci.trafficlights.getPhase(x)

3

In [30]:
counter = 0
while counter < 100:
    traci.simulationStep()
    if traci.trafficlights.getPhase(x) != prev_phase:
        print "Change! at step", step
        print "{} --> {}".format(prev_phase, traci.trafficlights.getPhase(x))
        prev_phase = traci.trafficlights.getPhase(x)
        if prev_phase == 0:
            print "Phase 0!"
            break
    step += 1
    counter += 1

Change! at step 599
9 --> 10
Change! at step 603
10 --> 11
Change! at step 607
11 --> 0
Phase 0!


In [31]:
traci.trafficlights.getPhase(x)

0

In [27]:
traci.trafficlights.setPhaseDuration(x, 100)

In [32]:
traci.trafficlights.getPhaseDuration(x)

31000

In [13]:
x = lr_1_dtse

In [16]:
max_vel = 0
for (vehicle_present, vehicle_vel, vehicle_id) in x:
    max_vel = max(max_vel, vehicle_vel)

In [17]:
max_vel

4.0

In [23]:
normalized_dtse = [[vehicle_present, (vehicle_vel/max_vel), vehicle_id] for (vehicle_present, vehicle_vel, vehicle_id) in x]

In [24]:
normalized_dtse

[[-1, 0.0, -999],
 [-1, 0.0, -999],
 [-1, 0.0, -999],
 [-1, 0.0, -999],
 [-1, 0.0, -999],
 [1, 1.0, 'type1.0'],
 [-1, 0.0, -999],
 [-1, 0.0, -999],
 [-1, 0.0, -999],
 [-1, 0.0, -999]]

In [27]:
def fun(x):
    x = x + 1
    return x

In [28]:
a = 5
for i in range(5):
    print fun(a)

6
6
6
6
6
